In [ ]:
from mlp.test_methods import test_dropout_layer
import numpy as np

fprop_test, fprop_output, fprop_correct, \
bprop_test, bprop_output, bprop_correct = test_dropout_layer()

assert fprop_test == 1.0, (
'The dropout layer fprop functionality test failed'
'Correct output is \n\n{0}\n\n but returned output is \n\n{1}\n\n difference is \n\n{2}'
.format(fprop_correct, fprop_output, fprop_output-fprop_correct)
)

print("Dropout Layer Fprop Functionality Test Passed")

assert bprop_test == 1.0, (
'The dropout layer bprop functionality test failed'
'Correct output is \n\n{0}\n\n but returned output is \n\n{1}\n\n difference is \n\n{2}'
.format(bprop_correct, bprop_output, bprop_output-bprop_correct)
)

print("Dropout Layer Bprop Test Passed")

In [ ]:
import numpy as np
import os
from mlp.layers import StochasticLayer


class DropoutLayer(StochasticLayer):
    def __init__(self, rng=None, incl_prob=0.5, share_across_batch=True):
        super(DropoutLayer, self).__init__(rng)
        assert incl_prob > 0. and incl_prob <= 1.
        self.incl_prob = incl_prob
        self.share_across_batch = share_across_batch
        self.rng = rng

    def fprop(self, inputs, stochastic=True):
        p = (self.rng.uniform(size = inputs.shape) <self.incl_prob)
        p = np.multiply(p,inputs)
        print('FPROP')
        return p
        

    def bprop(self, inputs, outputs, grads_wrt_outputs):
        return np.multiply(grads_wrt_outputs, (outputs!=0).astype(float))
    
    def __repr__(self):
        return 'DropoutLayer(incl_prob={0:.1f})'.format(self.incl_prob)


In [ ]:
# rng = np.random.RandomState(92019)
# x = rng.normal(loc=0, scale=5.0, size=(50, 3, 64, 64))
# correct_outputs = np.load(os.path.join(os.environ['MLP_DATA_DIR'], 'regularization_debug_pack.npy'), allow_pickle=True).item()
# rng = np.random.RandomState(92019)
# layer = DropoutLayer(rng=rng)
# out = layer.fprop(x)
# grads = layer.bprop(inputs=x, outputs=out, grads_wrt_outputs=np.ones(x.shape))
# fprop_test = np.allclose(correct_outputs['DropoutLayer_fprop'], out)
# bprop_test = np.allclose(correct_outputs['DropoutLayer_bprop'], grads)
# fprop_test, bprop_test

In [ ]:
rng = np.random.RandomState(92019)
x = rng.normal(loc=0, scale=5.0, size=(50, 3, 64, 64))
correct_outputs = np.load(os.path.join(os.environ['MLP_DATA_DIR'], 'regularization_debug_pack.npy'), allow_pickle=True).item()
rng = np.random.RandomState(92019)
layer = DropoutLayer(rng=rng)
out = layer.fprop(x)
grads = layer.bprop(inputs=x, outputs=out, grads_wrt_outputs=np.ones(x.shape))
true_outs = correct_outputs['DropoutLayer_fprop']
true_grads = correct_outputs['DropoutLayer_bprop']
fprop_test = np.allclose(true_outs, out)
bprop_test = np.allclose(true_grads, grads)
print(fprop_test,fprop_test)

# grads = (true_outs!=0).astype(float)
# print(np.allclose(true_grads, grads))

a =out.flatten()
b =true_outs.flatten()
g = []
k = 10
switch = True
old = -1
for i in range(0, 50*3*64*64, 1):
    if not np.allclose(a[i:i+5], b[i:i+5]) and switch:
        switch = False
        g.append(i)
        print(f' i = {i} : Adding for the first time {g} and switch is now {switch}') 
        old = i
        continue
    
    if not np.allclose(a[i:i+5], b[i:i+5]) and not switch:
        switch = False
        g.append(i)
        if not i == old+1:
            print(f' i = {i} old = {old} => {i == old+1}: Adding  for the {len(g)} time and switch is now {switch}')
        old = i
        
    if len(g) ==k:
        break
print(g[:k])